<a href="https://colab.research.google.com/github/slvrsrfrrr/Machine-Learning/blob/main/mRNA_degradation_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deep Learning Model to Predict mRNA Degregation

* Vaccines must be kept at low temperatures due to instabliity in mRNA molecule.

* This project will aim to forcast the rate of degredation.
* Degredation dataset from Kaggle


In [1]:
import json
import pandas as pd
import numpy as np

#visualisation

import plotly.express as px

#deep learning libraries

import tensorflow as tf
from tensorflow import keras
import keras.layers as L
from sklearn.model_selection import train_test_split

#seeds

tf.random.set_seed(2023)
np.random.seed(2023)

bring in data from kaggle:

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sefahmed","key":"8197c46f8100affcc3e1e2a341e604a9"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c stanford-covid-vaccine

 98% 88.0M/89.8M [00:05<00:00, 23.8MB/s]
100% 89.8M/89.8M [00:05<00:00, 16.8MB/s]


In [ ]:
!unzip /content/stanford-covid-vaccine.zip -d /content/data

# Data Descriptions

**id**
* An arbitrary identifier for each sample.

**seq_scored**
* (68 in Train and Public Test, 91 in Private Test)
* Integer value denoting the number of positions used in scoring with predicted values.
* This should match the length of reactivity, deg_* and *_error_* columns.
* Note that molecules used for the Private Test will be longer than those in the Train and Public Test data, so the size of this vector will be different.

**seq_length**
* (107 in Train and Public Test, 130 in Private Test)
* Integer values, denotes the length of sequence.
* Note that molecules used for the Private Test will be longer than those in the Train and Public Test data, so the size of this vector will be different.

**sequence**
* (1x107 string in Train and Public Test, 130 in Private Test)
* Describes the RNA sequence, a combination of A, G, U, and C for each sample.
* Should be 107 characters long, and the first 68 bases should correspond to the 68 positions specified in seq_scored (note: indexed starting at 0).

**structure**
* (1x107 string in Train and Public Test, 130 in Private Test)
* An array of (, ), and . characters that describe whether a base is estimated to be paired or unpaired.
* Paired bases are denoted by opening and closing parentheses e.g. (….) means that base 0 is paired to base 5, and bases 1-4 are unpaired.

**reactivity**
* (1x68 vector in Train and Public Test, 1x91 in Private Test)
* An array of floating point numbers, should have the same length as seq_scored.
* These numbers are reactivity values for the first 68 bases as denoted in sequence, and used to determine the likely secondary structure of the RNA sample.

**deg_pH10**
* (1x68 vector in Train and Public Test, 1x91 in Private Test)
* An array of floating point numbers, should have the same length as seq_scored.
* These numbers are reactivity values for the first 68 bases as denoted in sequence, and used to determine the likelihood of degradation at the base/linkage after incubating without magnesium at high pH (pH 10).

**deg_Mg_pH10**
* (1x68 vector in Train and Public Test, 1x91 in Private Test)
* An array of floating point numbers, should have the same length as seq_scored.
* These numbers are reactivity values for the first 68 bases as denoted in sequence, and used to determine the likelihood of degradation at the base/linkage after incubating with magnesium in high pH (pH 10).

**deg_50C**
* (1x68 vector in Train and Public Test, 1x91 in Private Test)
* An array of floating point numbers, should have the same length as seq_scored.
* These numbers are reactivity values for the first 68 bases as denoted in sequence, and used to determine the likelihood of degradation at the base/linkage after incubating without magnesium at high temperature (50 degrees Celsius).

**deg_Mg_50C**
* (1x68 vector in Train and Public Test, 1x91 in Private Test)
* An array of floating point numbers, should have the same length as seq_scored.
* These numbers are reactivity values for the first 68 bases as denoted in sequence, and used to determine the likelihood of degradation at the base/linkage after incubating with magnesium at high temperature (50 degrees Celsius).

*_error_*
* An array of floating point numbers, should have the same length as the corresponding reactivity or deg_* columns, calculated errors in experimental values obtained in reactivity and deg_* columns.

**predicted_loop_type**
* (1x107 string)
* Describes the structural context (also referred to as 'loop type')of each character in sequence.
* Loop types assigned by bpRNA from Vienna RNAfold 2 structure.

**target columns**
* reactivity, deg_Mg_pH10, deg_Mg_50C, deg_pH10, deg_50C

In [8]:
train = pd.read_json('/content/data/train.json', lines=True)
test = pd.read_json('/content/data/test.json', lines=True)
sample_df = pd.read_csv('/content/data/sample_submission.csv')

target_cols = ['reactivity','deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']
Model_Train = True

switching text to numerical.

# Explore Data

In [9]:
train.shape

(2400, 19)

In [10]:
test.shape

(3634, 7)

In [11]:
train.head(5)

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


In [12]:
fig = px.histogram(train, 'signal_to_noise', nbins=30, title='Signal to Noise Distr.', width=1000,height=500)
fig.show()

need to remove negative values.

In [13]:
train = train.query('signal_to_noise >=1')

look at sequence length distribution

In [14]:
fig = px.histogram(test, 'seq_length', nbins=25, title='sequence length dist.', width=1000,height=500)
fig.show()

two distinct sequence length values. could be a good idea to split the dataset and see if it improves the performance.

In [15]:
public_df = test.query('seq_length == 107')
private_df = test.query('seq_length == 130')

## Preprocessing

dictionary with character to integer tokenization

In [16]:
token2int = {x: i for i,x in enumerate('().ACGUBEHIMSX')}
token2int

{'(': 0,
 ')': 1,
 '.': 2,
 'A': 3,
 'C': 4,
 'G': 5,
 'U': 6,
 'B': 7,
 'E': 8,
 'H': 9,
 'I': 10,
 'M': 11,
 'S': 12,
 'X': 13}

now build a function that converts a dataframe into a 3D array

In [17]:
def dataframe3D(df):
  return np.transpose(np.array(df.values.tolist()), (0,2,1))

function that uses dictionary to convert training features into integer arrays, then convert that into a 3D array

In [18]:
def dataframe_encoding(df, token2int, cols=['sequence','structure','predicted_loop_type']):

  #tokenization of sequence, structure and predicted loop type

  return dataframe3D(df[cols].applymap(lambda seq: [token2int[x] for x in seq]))

In [19]:
train_inputs = dataframe_encoding(train,token2int)

train_labels = dataframe3D(train[target_cols])

split test set into public and private catagories

In [20]:
public_inputs = dataframe_encoding(public_df, token2int)
private_inputs = dataframe_encoding(private_df,token2int)

### build model

In [21]:
x_train, x_val, y_train, y_val = train_test_split(train_inputs, train_labels, test_size=0.1, random_state=42, stratify=train.SN_filter)

MC root mean squared error function

In [22]:
def MCRMSE(y_true,y_pred):
  colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
  return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

In [23]:
def build_model(embed_size,
                seq_len     =107,
                pred_length =68,
                dropout     = 0.5,
                sp_dropout  =0.2,
                embed_dim   =200,
                hidden_dim  =256):
  inputs = L.Input(shape=(seq_len, 3))
  embed = L.Embedding(input_dim=embed_size, output_dim=embed_dim)(inputs)
  reshape = tf.reshape(embed, shape=(-1,embed.shape[1], embed.shape[2]* embed.shape[3]))
  hidden = L.SpatialDropout1D(sp_dropout)(reshape)

  #three bi directional GRU layers

  hidden= L.Bidirectional(L.GRU(
    hidden_dim,
    dropout              = dropout,
    return_sequences    = True,
    kernel_initializer  = 'orthogonal',))(hidden)

  hidden= L.Bidirectional(L.GRU(
    hidden_dim,
    dropout              = dropout,
    return_sequences    = True,
    kernel_initializer  = 'orthogonal',))(hidden)

  hidden= L.Bidirectional(L.GRU(
    hidden_dim,
    dropout              = dropout,
    return_sequences    = True,
    kernel_initializer  = 'orthogonal',))(hidden)

  truncated = hidden[:,:pred_length]
  out = L.Dense(5, activation='linear')(truncated)
  model = tf.keras.Model(inputs = inputs, outputs = out)
  model.compile(optimizer='Adam', loss=MCRMSE)
  return model

In [24]:
model = build_model(embed_size=len(token2int))

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 107, 3)]          0         
                                                                 
 embedding (Embedding)       (None, 107, 3, 200)       2800      
                                                                 
 tf.reshape (TFOpLambda)     (None, 107, 600)          0         
                                                                 
 spatial_dropout1d (Spatial  (None, 107, 600)          0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 107, 512)          1317888   
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 107, 512)          118272

In [25]:
if Model_Train:
  history=model.fit(
      x_train,
      y_train,
      validation_data=(x_val,y_val),
      batch_size=64,
      epochs=40,
      verbose=2,
      callbacks=[tf.keras.callbacks.ReduceLROnPlateau(patience=5),
                 tf.keras.callbacks.ModelCheckpoint('Model/model.h5')]
  )

Epoch 1/40
30/30 - 21s - loss: 0.4579 - val_loss: 0.3915 - lr: 0.0010 - 21s/epoch - 684ms/step
Epoch 2/40


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



30/30 - 2s - loss: 0.3861 - val_loss: 0.3671 - lr: 0.0010 - 2s/epoch - 81ms/step
Epoch 3/40
30/30 - 2s - loss: 0.3656 - val_loss: 0.3504 - lr: 0.0010 - 2s/epoch - 79ms/step
Epoch 4/40
30/30 - 2s - loss: 0.3490 - val_loss: 0.3394 - lr: 0.0010 - 2s/epoch - 80ms/step
Epoch 5/40
30/30 - 2s - loss: 0.3372 - val_loss: 0.3258 - lr: 0.0010 - 2s/epoch - 82ms/step
Epoch 6/40
30/30 - 3s - loss: 0.3302 - val_loss: 0.3194 - lr: 0.0010 - 3s/epoch - 85ms/step
Epoch 7/40
30/30 - 2s - loss: 0.3241 - val_loss: 0.3110 - lr: 0.0010 - 2s/epoch - 80ms/step
Epoch 8/40
30/30 - 3s - loss: 0.3159 - val_loss: 0.3026 - lr: 0.0010 - 3s/epoch - 83ms/step
Epoch 9/40
30/30 - 3s - loss: 0.3091 - val_loss: 0.2963 - lr: 0.0010 - 3s/epoch - 88ms/step
Epoch 10/40
30/30 - 3s - loss: 0.3041 - val_loss: 0.2878 - lr: 0.0010 - 3s/epoch - 89ms/step
Epoch 11/40
30/30 - 3s - loss: 0.2950 - val_loss: 0.2791 - lr: 0.0010 - 3s/epoch - 86ms/step
Epoch 12/40
30/30 - 2s - loss: 0.2886 - val_loss: 0.2743 - lr: 0.0010 - 2s/epoch - 83ms/s

In [26]:
if Model_Train:
  fig=px.line(history.history,
              y=['loss', 'val_loss'],
              labels={'index': 'epoch', 'value': 'MCRMSE'},
              title='History')
  fig.show()

In [28]:
model_public = build_model(seq_len=107,
                           pred_length=107,
                           embed_size=len(token2int))
model_private = build_model(seq_len=130,
                           pred_length=130,
                           embed_size=len(token2int))

model_public.load_weights('Model/model.h5')
model_private.load_weights('Model/model.h5')

In [29]:
public_preds = model_public.predict(public_inputs)
private_preds = model_private.predict(private_inputs)

private_preds.shape

94/94 [==============================] - 4s 24ms/step


(3005, 130, 5)

In [30]:
preds_ls =[]

for df, preds in [(public_df, public_preds),(private_df, private_preds)]:
  for i, uid in enumerate(df.id):
    single_pred = preds[i]
    single_df = pd.DataFrame(single_pred, columns=target_cols)
    single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
    preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)

In [31]:
preds_df.head(5)

,reactivity,deg_Mg_pH10,deg_Mg_50C,deg_pH10,deg_50C,id_seqpos
0,0.590815,0.601068,0.531644,1.935872,0.798182,id_00073f8be_0
1,2.057342,3.107803,3.324284,4.364848,2.925877,id_00073f8be_1
2,1.465390,0.584802,0.581927,0.577345,0.710409,id_00073f8be_2
3,1.274872,1.353190,1.856409,1.230263,1.788942,id_00073f8be_3
4,0.883626,0.585739,0.852780,0.510890,0.907797,id_00073f8be_4
